In [21]:
import pandas as pd

pd.set_option("display.max_columns",200)
pd.set_option("display.max_rows",200)

In [24]:
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from sklearn import svm, linear_model, preprocessing
from sklearn.preprocessing import LabelBinarizer

import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import MySQLdb
import itertools

i = 0
X_train = []
Y_train = []
X_test = []
Y_test = []

# データ読み込み
conn = MySQLdb.connect(host='localhost', port=3306, user='user1', passwd='Silver8810', db='hrdb', charset='utf8')
cur = conn.cursor()
cur.execute("SELECT Frame, HName, Sex, Age, Handicap, RName, Weight, Increase, Year, Month, Start, Weather, Ground, GCondition, Distance, Prize1, Prize2, Prize3, Prize4, Prize5 from jra_data")
for Var in cur:
    if i in range(2500):
        X_train.append(Var)
        i += 1
    else:
        X_test.append(Var)
        i += 1
cur.execute("SELECT Pref from jra_data")
i =0
for Var in cur:
    if i in range(2500):
        i += 1
        if Var[0] < 4:
            Y_train.append(1)
        else:
            Y_train.append(0)
    else:
        i += 1
        if Var[0] < 4:
            Y_test.append(1)
        else:
            Y_test.append(0)

# One-hot encording
# Xに入れる際にそれぞれの要素内で順番が逆になっている。 print(Sex_ohe[0]と実際にX内に入っている数値が逆)forの仕様？
# 全て一律に逆になっているのであれば問題はないので修正はしない。
Frame_ohe = [i[0] for i in X_train]
Frame_ohe = LabelBinarizer().fit_transform(Frame_ohe)
HName_ohe = [i[1] for i in X_train]
HName_ohe = LabelBinarizer().fit_transform(HName_ohe)
Sex_ohe = [i[2] for i in X_train]
Sex_ohe = LabelBinarizer().fit_transform(Sex_ohe)
RName_ohe = [i[5] for i in X_train]
RName_ohe = LabelBinarizer().fit_transform(RName_ohe)
Year_ohe = [i[8] for i in X_train]
Year_ohe = LabelBinarizer().fit_transform(Year_ohe)
Month_ohe = [i[9] for i in X_train]
Month_ohe = LabelBinarizer().fit_transform(Month_ohe)
Start_ohe = [i[10] for i in X_train]
Start_ohe = LabelBinarizer().fit_transform(Start_ohe)
Weather_ohe = [i[11] for i in X_train]
Weather_ohe = LabelBinarizer().fit_transform(Weather_ohe)
Ground_ohe = [i[12] for i in X_train]
Ground_ohe = LabelBinarizer().fit_transform(Ground_ohe)
GCondition_ohe = [i[13] for i in X_train]
GCondition_ohe = LabelBinarizer().fit_transform(GCondition_ohe)

X_train = np.delete(X_train,slice(0,3),1)
X_train = np.delete(X_train,2,1)
X_train = np.delete(X_train,slice(4,10),1)
print(len(X_train))

for i in range(len(X_train)):
    for j in Frame_ohe[i]:
        X_train[i].insert(0, j)
    for j in HName_ohe[i]:
        X_train[i].insert(18, j)
    for j in Sex_ohe[i]:
        X_train[i].insert(1322, j)
    for j in RName_ohe[i]:
        X_train[i].insert(1328, j)
    for j in Year_ohe[i]:
        X_train[i].insert(1494, j)
    for j in Month_ohe[i]:
        X_train[i].insert(1495, j)
    for j in Start_ohe[i]:
        X_train[i].insert(1507, j)
    for j in Weather_ohe[i]:
        X_train[i].insert(1513, j)
    for j in Ground_ohe[i]:
        X_train[i].insert(1518, j)
    for j in GCondition_ohe[i]:
        X_train[i].insert(1521, j)

X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)
print(X_train)


# Yは0と1からなるリスト
#Y = to_categorical(Y)

# モデル
model = Sequential()
# 全結合層(18を500に)
#model.add(Dense(input_dim=100, output_dim=500))
#model.add(Dense(units=20, input_dim=100, output_dim=500))
model.add(Dense(100, activation='relu',input_dim=20))
# 各バッチ毎に前の層の出力を正規化する。
model.add(BatchNormalization())
# 全結合層(500を2に) 
#model.add(Dense(output_dim=2))
model.add(Dense(1,activation = 'softmax'))
# softmax関数
#model.add(Activation("softmax"))
# コンパイル
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
# 実行
#model.fit(X, Y, nb_epoch=300, batch_size=50,validation_split=0.1)
model.fit(X_train, Y_train, epochs=300, batch_size=50,validation_split=0.1)
loss_and_metrics = model.evaluate(X_test, Y_test, batch_size=128)

# 予測
results = model.predict_proba(np.array(X_test))
# 結果
print("Predict:\n", results)

# グラフ化
#plt.plot(X, Y)
#plt.grid(True)
#plt.show()

#cur.close()
#conn.close()

2500


AttributeError: 'numpy.ndarray' object has no attribute 'insert'

In [17]:
import numpy as np

a = np.arange(24).reshape(4, 6)
print(a)
print(f"00={np.delete(a,0,0)}")
print(f"01={np.delete(a,0,1)}")
print(f"10={np.delete(a,1,0)}")
print(f"20={np.delete(a,2,0)}")
print(f"20={np.delete(a,slice(0,3),1)}")

[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]]
00=[[ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]]
01=[[ 1  2  3  4  5]
 [ 7  8  9 10 11]
 [13 14 15 16 17]
 [19 20 21 22 23]]
10=[[ 0  1  2  3  4  5]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]]
20=[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [18 19 20 21 22 23]]
20=[[ 3  4  5]
 [ 9 10 11]
 [15 16 17]
 [21 22 23]]


In [25]:
pip install --upgrade jupyterlab-git

Note: you may need to restart the kernel to use updated packages.
